# Covid-19 (Coronavirus): Data Extract, Transform, Load (ETL) Notebook
This notebook pulls data from various sources, performs data wrangling to get it into more usable formats, and exports to CSV for use in other notebooks, as well as for future reference.  

Feeds into the Covid19_Analytics notebook.

## Data Sources

### Primary Data Source: Johns Hopkins CSSE Data Repository 
- Link: https://github.com/CSSEGISandData/COVID-19  
- **Live data:**
    - Countries (cases, deaths, recoveries, active)
    - US States (cases, deaths)
    - US County (cases, deaths)
- **Historic time series data:**
    - Countries (cases, deaths, recoveries, active)
    - US States (cases, deaths)
    - US Counties (cases, deaths)

### US State Testing and Hospitalizations: Covid Tracking Project 
- Link: https://covidtracking.com/data/ 
- **Live data:**
    - US State testing and hospitalization, ICU stats
- **Historic time series data:**
    - US State testing and hospitalization, ICU stats

## Import Libraries

In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from datetime import datetime
import os

## Get the Datasets

### Dataset 1: John Hopkins

Live data is stored in files named MM-DD-YYYY.csv, so get the current date for live data updates.  
https://github.com/CSSEGISandData/COVID-19/tree/master/csse_covid_19_data/csse_covid_19_daily_reports

#### 1A: Pull the Data from JH's github repo, immediately export as 'orig'

In [2]:
# Create the filename to load based on today's date

day = str(datetime.now().day) if datetime.now().day >= 10 else "0" + str(datetime.now().day)
month = str(datetime.now().month) if datetime.now().month >= 10 else "0" + str(datetime.now().month)
year = str(datetime.now().year)

jh_live_date = month + "-" + day + "-" + year
jh_live_date

'07-27-2021'

In [3]:
#  First try to pull today's data, but if it's not available yet, pull yesterday's data
try:
    fileURL = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/" + jh_live_date + ".csv"
    jh_live_global = pd.read_csv(fileURL)
except:
    day = str(datetime.now().day-1) if datetime.now().day-1 >= 10 else "0" + str(datetime.now().day-1)
    jh_live_date = month + "-" + day + "-" + year
    fileURL = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/" + jh_live_date + ".csv"
    jh_live_global = pd.read_csv(fileURL)
    
jh_live_date

'07-27-2021'

In [4]:
# Check out the first x rows to make sure the data loaded correctly
jh_live_global.head()

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incident_Rate,Case_Fatality_Ratio
0,NaN,NaN,NaN,Afghanistan,2021-07-28 04:21:39,33.93911,67.709953,145008,6515,82586.0,55907.0,Afghanistan,372.499820,4.492856
1,NaN,NaN,NaN,Albania,2021-07-28 04:21:39,41.15330,20.168300,132922,2456,130166.0,300.0,Albania,4618.875530,1.847700
2,NaN,NaN,NaN,Algeria,2021-07-28 04:21:39,28.03390,1.659600,165204,4112,112050.0,49042.0,Algeria,376.739044,2.489044
3,NaN,NaN,NaN,Andorra,2021-07-28 04:21:39,42.50630,1.521800,14586,127,14113.0,346.0,Andorra,18877.887789,0.870698
4,NaN,NaN,NaN,Angola,2021-07-28 04:21:39,-11.20270,17.873900,42110,994,35742.0,5374.0,Angola,128.125286,2.360484


Now immediately export the original jh dataset before making any changes

In [5]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_live_global_orig.csv')
jh_live_global.to_csv(filename, index=False)

**Also pull the global historic time series data.  Note that it won't have live data from today until close to midnight PT.**

In [6]:
jh_hist_global_cases = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv')
jh_hist_global_cases.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,7/18/21,7/19/21,7/20/21,7/21/21,7/22/21,7/23/21,7/24/21,7/25/21,7/26/21,7/27/21
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,137853,141489,142414,142414,143183,143439,143439,143439,144285,145008
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,132686,132697,132740,132763,132797,132828,132853,132875,132891,132922
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,153309,154486,155784,157005,158213,159563,160868,162155,163660,165204


In [7]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_cases_orig.csv')
jh_hist_global_cases.to_csv(filename, index=False)

In [8]:
jh_hist_global_deaths = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv')
jh_hist_global_deaths.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,7/18/21,7/19/21,7/20/21,7/21/21,7/22/21,7/23/21,7/24/21,7/25/21,7/26/21,7/27/21
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,5983,6213,6266,6266,6325,6357,6357,6357,6477,6515
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,2456,2456,2456,2456,2456,2456,2456,2456,2456,2456
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,3938,3956,3979,3994,4008,4026,4042,4063,4087,4112


In [9]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_deaths_orig.csv')
jh_hist_global_deaths.to_csv(filename, index=False)

In [10]:
jh_hist_global_recovered = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv')
jh_hist_global_recovered.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,7/18/21,7/19/21,7/20/21,7/21/21,7/22/21,7/23/21,7/24/21,7/25/21,7/26/21,7/27/21
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,82586,82586,82586,82586,82586,82586,82586,82586,82586,82586
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,130074,130081,130086,130097,130109,130118,130125,130139,130152,130166
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,105604,106337,107041,107776,108537,109349,109951,110577,111322,112050


In [11]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_recovered_orig.csv')
jh_hist_global_recovered.to_csv(filename, index=False)

In [12]:
jh_hist_uscounties_cases = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_US.csv')
jh_hist_uscounties_cases.head(3)

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,7/18/21,7/19/21,7/20/21,7/21/21,7/22/21,7/23/21,7/24/21,7/25/21,7/26/21,7/27/21
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,7347,7350,7365,7375,7388,7394,7402,7419,7435,7456
1,84001003,US,USA,840,1003.0,Baldwin,Alabama,US,30.727750,-87.722071,...,22802,22876,23032,23184,23319,23487,23668,23810,23922,24213
2,84001005,US,USA,840,1005.0,Barbour,Alabama,US,31.868263,-85.387129,...,2394,2396,2411,2415,2418,2422,2431,2434,2437,2451


In [13]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_uscounties_cases_orig.csv')
jh_hist_uscounties_cases.to_csv(filename, index=False)

In [14]:
jh_hist_uscounties_deaths = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_US.csv')
jh_hist_uscounties_deaths.head(3)

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,7/18/21,7/19/21,7/20/21,7/21/21,7/22/21,7/23/21,7/24/21,7/25/21,7/26/21,7/27/21
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,113,113,113,113,113,113,113,113,113,114
1,84001003,US,USA,840,1003.0,Baldwin,Alabama,US,30.727750,-87.722071,...,323,323,324,324,324,324,325,325,325,326
2,84001005,US,USA,840,1005.0,Barbour,Alabama,US,31.868263,-85.387129,...,60,60,60,60,60,60,61,61,61,61


In [15]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_uscounties_deaths_orig.csv')
jh_hist_uscounties_deaths.to_csv(filename, index=False)

#### 1B: Wrangle the data into more usable format

For the JH live global data, it will be useful to group / subtotal by Country, by US State, and by US County.  

For the historic time series data, these need to be rearranged so that the date is in 
datetime format and pivoted so the date is the index of the DataFrame.

In [16]:
jh_live_global[jh_live_global['Country_Region']=='US'].sort_values('Province_State').head(3)

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incident_Rate,Case_Fatality_Ratio
656,1001.0,Autauga,Alabama,US,2021-07-28 04:21:39,32.539527,-86.644082,7456,114,NaN,NaN,"Autauga, Alabama, US",13345.504663,1.528970
693,1075.0,Lamar,Alabama,US,2021-07-28 04:21:39,33.779950,-88.096680,1497,38,NaN,NaN,"Lamar, Alabama, US",10843.897139,2.538410
694,1077.0,Lauderdale,Alabama,US,2021-07-28 04:21:39,34.901719,-87.656247,10044,253,NaN,NaN,"Lauderdale, Alabama, US",10831.562941,2.518917


##### 1B-1: Pivot Table on the JH live global data, grouped / subtotalled by **Country**

In [17]:
jh_live_countries = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Confirmed', 'Deaths', 'Recovered'], aggfunc=np.sum)

In [18]:
jh_live_countries.sort_values('Confirmed', ascending=False).head(3)

,Confirmed,Deaths,Recovered
Country_Region,,,
US,34603919,611414,0.0
India,31440951,421382,30621469.0
Brazil,19749073,551835,17715481.0


In [19]:
jh_live_countries['Active'] = jh_live_countries['Confirmed'] - jh_live_countries['Deaths'] - jh_live_countries['Recovered']

In [20]:
jh_live_countries['Last_Update'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Last_Update'], aggfunc=np.max)
jh_live_countries['Lat'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Lat'], aggfunc=np.mean)
jh_live_countries['Long'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Long_'], aggfunc=np.mean)

In [21]:
jh_live_countries.sort_values('Confirmed', ascending=False).head(5)

,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long
Country_Region,,,,,,,
US,34603919,611414,0.0,33992505.0,2021-07-28 04:21:39,37.950547,-91.419547
India,31440951,421382,30621469.0,398100.0,2021-07-28 04:21:39,23.088275,81.806127
Brazil,19749073,551835,17715481.0,1481757.0,2021-07-28 04:21:39,-12.669522,-48.480493
Russia,6094379,152836,5459932.0,481611.0,2021-07-28 04:21:39,54.546312,62.120860
France,6088930,111883,413226.0,5563821.0,2021-07-28 04:21:39,6.430808,-34.730285


##### 1B-2: Pivot Table on the JH live global data, grouped / subtotalled by **US State**

In [22]:
jh_live_usstates = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Confirmed', 'Deaths', 'Recovered'], 
                                    aggfunc=np.sum)

In [23]:
jh_live_usstates.sort_values('Confirmed', ascending=False).head(3)

,Confirmed,Deaths,Recovered
Province_State,,,
California,3927901,64292,0.0
Texas,3095920,53078,0.0
Florida,2523510,38670,0.0


In [24]:
jh_live_usstates['Active'] = jh_live_usstates['Confirmed'] - jh_live_usstates['Deaths'] - jh_live_usstates['Recovered']

In [25]:
jh_live_usstates['Last_Update'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Last_Update'], 
                                    aggfunc=np.max)
jh_live_usstates['Lat'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Lat'], 
                                    aggfunc=np.mean)
jh_live_usstates['Long'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Long_'], 
                                    aggfunc=np.mean)

In [26]:
jh_live_usstates.sort_values('Confirmed', ascending=False).head(5)

,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long
Province_State,,,,,,,
California,3927901,64292,0.0,3863609.0,2021-07-28 04:21:39,37.843962,-120.728594
Texas,3095920,53078,0.0,3042842.0,2021-07-28 04:21:39,31.660643,-98.653069
Florida,2523510,38670,0.0,2484840.0,2021-07-28 04:21:39,28.940755,-82.700744
New York,2140258,53611,0.0,2086647.0,2021-07-28 04:21:39,42.544151,-75.474183
Illinois,1413490,25847,0.0,1387643.0,2021-07-28 04:21:39,39.843618,-89.178330


##### 1B-3: Pivot Table on the JH live global data, grouped / subtotalled by **US County**

In [27]:
jh_live_uscounties = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Confirmed', 'Deaths', 'Recovered'], 
                                    aggfunc=np.sum)

In [28]:
jh_live_uscounties.sort_values('Confirmed', ascending=False).head(5)

,,Confirmed,Deaths,Recovered
Province_State,Admin2,,,
California,Los Angeles,1288090,24647,0.0
Arizona,Maricopa,577342,10422,0.0
Illinois,Cook,562540,10557,0.0
Florida,Miami-Dade,533821,6472,0.0
Texas,Harris,414098,6691,0.0


In [29]:
jh_live_uscounties['Active'] = jh_live_uscounties['Confirmed'] - jh_live_uscounties['Deaths'] - jh_live_uscounties['Recovered']


In [30]:
jh_live_uscounties['Last_Update'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Last_Update'], 
                                    aggfunc=np.max)
jh_live_uscounties['Lat'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Lat'], 
                                    aggfunc=np.mean)
jh_live_uscounties['Long'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Long_'], 
                                    aggfunc=np.mean)
jh_live_uscounties['FIPS'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['FIPS'], 
                                    aggfunc=np.max)

In [31]:
jh_live_uscounties.sort_values('Confirmed', ascending=False).head(5)

,,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long,FIPS
Province_State,Admin2,,,,,,,,
California,Los Angeles,1288090,24647,0.0,1263443.0,2021-07-28 04:21:39,34.308284,-118.228241,6037.0
Arizona,Maricopa,577342,10422,0.0,566920.0,2021-07-28 04:21:39,33.348359,-112.491815,4013.0
Illinois,Cook,562540,10557,0.0,551983.0,2021-07-28 04:21:39,41.841448,-87.816588,17031.0
Florida,Miami-Dade,533821,6472,0.0,527349.0,2021-07-28 04:21:39,25.611236,-80.551706,12086.0
Texas,Harris,414098,6691,0.0,407407.0,2021-07-28 04:21:39,29.858649,-95.393395,48201.0


##### 1B-4: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [32]:
# First, isolate the date columns of the JH historic time series data to be used in the following 3 Pivot Tables
date_columns = jh_hist_global_cases.iloc[:, 4:].columns

In [33]:
jh_hist_countries_cases = pd.pivot_table(jh_hist_global_cases, index=['Country/Region'], 
                                            values=date_columns, aggfunc=np.sum)
jh_hist_countries_cases = jh_hist_countries_cases.transpose()
jh_hist_countries_cases.index = pd.to_datetime(jh_hist_countries_cases.index)
jh_hist_countries_cases = jh_hist_countries_cases.sort_index()

In [34]:
jh_hist_countries_cases.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-07-25,143439,132875,162155,14498,41780,1280,4846615,228798,33082,656270,...,5723399,380431,124290,4,299822,101173,316088,7012,192071,97894
2021-07-26,144285,132891,163660,14577,41879,1280,4859170,228910,33266,656582,...,5748331,380584,124995,4,300919,106347,316088,7012,192316,99944
2021-07-27,145008,132922,165204,14586,42110,1288,4875927,229090,33473,656949,...,5771732,380793,125784,4,301979,117121,316189,7022,192956,101711


##### 1B-5: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **Country**

In [35]:
jh_hist_countries_deaths = pd.pivot_table(jh_hist_global_deaths, index=['Country/Region'], values=date_columns, aggfunc=np.sum)
jh_hist_countries_deaths = jh_hist_countries_deaths.transpose()
jh_hist_countries_deaths.index = pd.to_datetime(jh_hist_countries_deaths.index)
jh_hist_countries_deaths = jh_hist_countries_deaths.sort_index()

In [36]:
jh_hist_countries_deaths.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-07-25,6357,2456,4063,127,984,43,103721,4587,918,10731,...,129446,5931,832,1,3492,370,3599,1373,3272,3094
2021-07-26,6477,2456,4087,127,991,43,104105,4590,922,10732,...,129460,5936,839,1,3509,524,3599,1373,3295,3173
2021-07-27,6515,2456,4112,127,994,43,104352,4597,921,10737,...,129591,5941,845,1,3524,524,3600,1374,3316,3280


##### 1B-6: Pivot Table on the JH historic **recoveries**, grouped / subtotalled by **Country**

In [37]:
jh_hist_countries_recovered = pd.pivot_table(jh_hist_global_recovered, index=['Country/Region'], values=date_columns, aggfunc=np.sum)
jh_hist_countries_recovered = jh_hist_countries_recovered.transpose()
jh_hist_countries_recovered.index = pd.to_datetime(jh_hist_countries_recovered.index)
jh_hist_countries_recovered = jh_hist_countries_recovered.sort_index()

In [38]:
jh_hist_countries_recovered.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-07-25,82586,130139,110577,13988,35474,1229,4493984,219062,24001,640951,...,18226,371822,118638,3,283215,19342,311526,4166,181638,65913
2021-07-26,82586,130152,111322,14077,35686,1229,4508137,219168,24008,641222,...,18546,372181,119449,3,284435,21344,311526,4166,182665,67827
2021-07-27,82586,130166,112050,14113,35742,1229,4519922,219280,24013,641541,...,18908,372448,120320,3,285776,22946,311560,4168,183534,70496


##### 1B-7: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [39]:
jh_hist_uscounties_cases.sort_values('Province_State').head()

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,7/18/21,7/19/21,7/20/21,7/21/21,7/22/21,7/23/21,7/24/21,7/25/21,7/26/21,7/27/21
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,7347,7350,7365,7375,7388,7394,7402,7419,7435,7456
37,84001075,US,USA,840,1075.0,Lamar,Alabama,US,33.779950,-88.096680,...,1485,1485,1486,1489,1491,1491,1494,1495,1495,1497
38,84001077,US,USA,840,1077.0,Lauderdale,Alabama,US,34.901719,-87.656247,...,9805,9813,9830,9872,9883,9918,9968,9991,10009,10044
39,84001079,US,USA,840,1079.0,Lawrence,Alabama,US,34.520415,-87.310695,...,3164,3164,3165,3170,3176,3183,3188,3192,3197,3211
40,84001081,US,USA,840,1081.0,Lee,Alabama,US,32.601549,-85.351322,...,16574,16587,16630,16666,16691,16722,16753,16801,16828,16897


In [40]:
date_columns = jh_hist_uscounties_cases.iloc[:,11:].columns

In [41]:
jh_hist_usstates_cases = pd.pivot_table(jh_hist_uscounties_cases, index=['Province_State'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_usstates_cases = jh_hist_usstates_cases.transpose()
jh_hist_usstates_cases.index = pd.to_datetime(jh_hist_usstates_cases.index)
jh_hist_usstates_cases = jh_hist_usstates_cases.sort_index()

In [42]:
jh_hist_usstates_cases.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2021-07-25,570667,73818,0,917168,375350,3905990,569289,352037,110797,49,...,880789,3077513,426418,24676,4375,688300,466235,165702,682077,64251
2021-07-26,572070,74383,0,918609,375971,3917610,571087,352626,110862,49,...,884417,3083368,428687,24748,4403,690183,469308,166108,683041,64451
2021-07-27,574737,74383,0,920084,378023,3927901,571958,353114,110943,49,...,886519,3095920,429300,24760,4450,691018,470333,166297,684119,64623


##### 1B-8: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [43]:
jh_hist_uscounties_cases = pd.pivot_table(jh_hist_uscounties_cases, index=['Province_State', 'Admin2'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_uscounties_cases = jh_hist_uscounties_cases.transpose()
jh_hist_uscounties_cases.index = pd.to_datetime(jh_hist_uscounties_cases.index)
jh_hist_uscounties_cases = jh_hist_uscounties_cases.sort_index()

In [44]:
jh_hist_uscounties_cases.tail(3)

Province_State Alabama                                                      \
Admin2         Autauga Baldwin Barbour  Bibb Blount Bullock Butler Calhoun   
2021-07-25        7419   23810    2434  2801   7184    1256   2361   15130   
2021-07-26        7435   23922    2437  2814   7195    1256   2365   15158   
2021-07-27        7456   24213    2451  2829   7233    1260   2376   15210   

Province_State                    ... Wyoming                           \
Admin2         Chambers Cherokee  ...    Park Platte Sheridan Sublette   
2021-07-25         3852     1939  ...    2940    739     3290      838   
2021-07-26         3857     1939  ...    2946    742     3297      842   
2021-07-27         3880     1945  ...    2955    752     3306      849   

Province_State                                                    
Admin2         Sweetwater Teton Uinta Unassigned Washakie Weston  
2021-07-25           4940  3868  2372          0      943    672  
2021-07-26           4955  3881  2382          0      944    677  
2021-07-27           4968  3883  2384          0      945    679  

[3 rows x 3336 columns]

##### 1B-9: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **US State**

In [45]:
jh_hist_usstates_deaths = pd.pivot_table(jh_hist_uscounties_deaths, index=['Province_State'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_usstates_deaths = jh_hist_usstates_deaths.transpose()
jh_hist_usstates_deaths.index = pd.to_datetime(jh_hist_usstates_deaths.index)
jh_hist_usstates_deaths = jh_hist_usstates_deaths.sort_index()

In [46]:
jh_hist_usstates_deaths.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2021-07-25,11483,384,0,18171,6054,64235,6910,8286,1698,0,...,12664,53018,2425,259,35,11500,6078,2936,8240,766
2021-07-26,11483,384,0,18171,6077,64267,6924,8288,1698,0,...,12683,53037,2434,259,35,11506,6089,2936,8253,766
2021-07-27,11492,384,0,18183,6087,64292,6922,8288,1698,0,...,12703,53078,2441,259,36,11515,6097,2936,8184,776


##### 1B-10: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **US County**

In [47]:
jh_hist_uscounties_deaths = pd.pivot_table(jh_hist_uscounties_deaths, index=['Province_State', 'Admin2'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_uscounties_deaths = jh_hist_uscounties_deaths.transpose()
jh_hist_uscounties_deaths.index = pd.to_datetime(jh_hist_uscounties_deaths.index)
jh_hist_uscounties_deaths = jh_hist_uscounties_deaths.sort_index()

In [48]:
jh_hist_uscounties_deaths.tail(3)

Province_State Alabama                                                     \
Admin2         Autauga Baldwin Barbour Bibb Blount Bullock Butler Calhoun   
2021-07-25         113     325      61   65    139      42     72     332   
2021-07-26         113     325      61   65    139      42     72     332   
2021-07-27         114     326      61   65    139      42     72     332   

Province_State                    ... Wyoming                           \
Admin2         Chambers Cherokee  ...    Park Platte Sheridan Sublette   
2021-07-25          125       48  ...      33     14       31        7   
2021-07-26          125       48  ...      33     14       31        7   
2021-07-27          125       48  ...      34     14       31        8   

Province_State                                                    
Admin2         Sweetwater Teton Uinta Unassigned Washakie Weston  
2021-07-25             43    11    13          0       26      6  
2021-07-26             43    11    13          0       26      6  
2021-07-27             44    11    14          0       26      6  

[3 rows x 3336 columns]

#### 1C: Export the data

##### 1C-1: Export the JH live global data, grouped / subtotalled by **Country**

In [49]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_countries.csv')
jh_live_countries.to_csv(filename, index=True)

##### 1C-2: Export the JH live global data, grouped / subtotalled by **US State**

In [50]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_usstates.csv')
jh_live_usstates.to_csv(filename, index=True)

##### 1C-3: Export the JH live global data, grouped / subtotalled by **US County**

In [51]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_uscounties.csv')
jh_live_uscounties.to_csv(filename, index=True)

##### 1C-4: Export the JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [52]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases.csv')
jh_hist_countries_cases.to_csv(filename, index=True)

##### 1C-5: Export the JH historic **deaths**, grouped / subtotalled by **Country**

In [53]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths.csv')
jh_hist_countries_deaths.to_csv(filename, index=True)

##### 1C-6: Export the JH historic **recoveries**, grouped / subtotalled by **Country**

In [54]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered.csv')
jh_hist_countries_recovered.to_csv(filename, index=True)

##### 1C-7: Export the JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [55]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases.csv')
jh_hist_usstates_cases.to_csv(filename, index=True)

##### 1C-8: Export the JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [56]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases.csv')
jh_hist_uscounties_cases.to_csv(filename, index=True)

##### 1C-9: Export the JH historic **deaths**, grouped / subtotalled by **US State**

In [57]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths.csv')
jh_hist_usstates_deaths.to_csv(filename, index=True)

##### 1C-10: Export the JH historic **deaths**, grouped / subtotalled by **US County**

In [58]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths.csv')
jh_hist_uscounties_deaths.to_csv(filename, index=True)

#### 1D: Calculate daily % change for each historic dataset (match numbering sequence, export as well)

##### 1D-4: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [59]:
jh_hist_countries_cases_pct = jh_hist_countries_cases.pct_change()

In [60]:
jh_hist_countries_cases_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-07-25,0.000000,0.000166,0.008000,0.000000,0.001054,0.00000,0.001551,0.000726,0.004982,0.000506,...,0.005082,0.000316,0.005916,0.0,0.003407,0.065955,0.00038,0.000571,0.002840,0.006343
2021-07-26,0.005898,0.000120,0.009281,0.005449,0.002370,0.00000,0.002590,0.000490,0.005562,0.000475,...,0.004356,0.000402,0.005672,0.0,0.003659,0.051140,0.00000,0.000000,0.001276,0.020941
2021-07-27,0.005011,0.000233,0.009434,0.000617,0.005516,0.00625,0.003449,0.000786,0.006223,0.000559,...,0.004071,0.000549,0.006312,0.0,0.003523,0.101310,0.00032,0.001426,0.003328,0.017680


In [61]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases_pct.csv')
jh_hist_countries_cases_pct.to_csv(filename, index=True)

##### 1D-5: Daily % change JH historic **deaths**, grouped / subtotalled by **Country**

In [62]:
jh_hist_countries_deaths_pct = jh_hist_countries_deaths.pct_change()

In [63]:
jh_hist_countries_deaths_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-07-25,0.000000,0.0,0.005195,0.0,0.002037,0.0,0.001323,0.000654,0.000000,0.000000,...,0.000216,0.000675,0.007264,0.0,0.004892,0.000000,0.000834,0.000000,0.006769,0.014426
2021-07-26,0.018877,0.0,0.005907,0.0,0.007114,0.0,0.003702,0.000654,0.004357,0.000093,...,0.000108,0.000843,0.008413,0.0,0.004868,0.416216,0.000000,0.000000,0.007029,0.025533
2021-07-27,0.005867,0.0,0.006117,0.0,0.003027,0.0,0.002373,0.001525,-0.001085,0.000466,...,0.001012,0.000842,0.007151,0.0,0.004275,0.000000,0.000278,0.000728,0.006373,0.033722


In [64]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths_pct.csv')
jh_hist_countries_deaths_pct.to_csv(filename, index=True)

##### 1D-6: Daily % change Export the JH historic **recoveries**, grouped / subtotalled by **Country**

In [65]:
jh_hist_countries_recovered_pct = jh_hist_countries_recovered.pct_change()

In [66]:
jh_hist_countries_recovered_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-07-25,0.0,0.000108,0.005693,0.000000,0.001440,0.0,0.003046,0.000475,0.000000,0.000415,...,0.000549,0.000966,0.005731,0.0,0.003593,0.100040,0.000469,0.00000,0.006110,0.019883
2021-07-26,0.0,0.000100,0.006737,0.006363,0.005976,0.0,0.003149,0.000484,0.000292,0.000423,...,0.017557,0.000966,0.006836,0.0,0.004308,0.103505,0.000000,0.00000,0.005654,0.029038
2021-07-27,0.0,0.000108,0.006540,0.002557,0.001569,0.0,0.002614,0.000511,0.000208,0.000497,...,0.019519,0.000717,0.007292,0.0,0.004715,0.075056,0.000109,0.00048,0.004757,0.039350


In [67]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered_pct.csv')
jh_hist_countries_recovered_pct.to_csv(filename, index=True)

##### 1D-7: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [68]:
jh_hist_usstates_cases_pct = jh_hist_usstates_cases.pct_change()

In [69]:
jh_hist_usstates_cases_pct.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2021-07-25,0.002699,0.000000,NaN,0.001647,0.002730,0.000753,0.000000,0.000000,0.000614,0.0,...,0.000000,0.000454,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2021-07-26,0.002459,0.007654,NaN,0.001571,0.001654,0.002975,0.003158,0.001673,0.000587,0.0,...,0.004119,0.001903,0.005321,0.002918,0.006400,0.002736,0.006591,0.002450,0.001413,0.003113
2021-07-27,0.004662,0.000000,NaN,0.001606,0.005458,0.002627,0.001525,0.001384,0.000731,0.0,...,0.002377,0.004071,0.001430,0.000485,0.010675,0.001210,0.002184,0.001138,0.001578,0.002669


In [70]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases_pct.csv')
jh_hist_usstates_cases_pct.to_csv(filename, index=True)

##### 1D-8: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [71]:
jh_hist_uscounties_cases_pct = jh_hist_uscounties_cases.pct_change()

In [72]:
jh_hist_uscounties_cases_pct.tail(3)

Province_State   Alabama                                                    \
Admin2           Autauga   Baldwin   Barbour      Bibb    Blount   Bullock   
2021-07-25      0.002297  0.006000  0.001234  0.002147  0.000278  0.000000   
2021-07-26      0.002157  0.004704  0.001233  0.004641  0.001531  0.000000   
2021-07-27      0.002824  0.012165  0.005745  0.005330  0.005281  0.003185   

Province_State                                          ...   Wyoming  \
Admin2            Butler   Calhoun  Chambers  Cherokee  ...      Park   
2021-07-25      0.002122  0.001522  0.002603  0.000516  ...  0.000000   
2021-07-26      0.001694  0.001851  0.001298  0.000000  ...  0.002041   
2021-07-27      0.004651  0.003431  0.005963  0.003094  ...  0.003055   

Province_State                                                               \
Admin2            Platte  Sheridan  Sublette Sweetwater     Teton     Uinta   
2021-07-25      0.000000  0.000000  0.000000   0.000000  0.000000  0.000000   
2021-07-26      0.004060  0.002128  0.004773   0.003036  0.003361  0.004216   
2021-07-27      0.013477  0.002730  0.008314   0.002624  0.000515  0.000840   

Province_State                                 
Admin2         Unassigned  Washakie    Weston  
2021-07-25            NaN  0.000000  0.000000  
2021-07-26            NaN  0.001060  0.007440  
2021-07-27            NaN  0.001059  0.002954  

[3 rows x 3336 columns]

In [73]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases_pct.csv')
jh_hist_uscounties_cases_pct.to_csv(filename, index=True)

##### 1D-9: Daily % change JH historic **deaths**, grouped / subtotalled by **US State**

In [74]:
jh_hist_usstates_deaths_pct = jh_hist_usstates_deaths.pct_change()

In [75]:
jh_hist_usstates_deaths_pct.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2021-07-25,0.000000,0.0,NaN,0.000055,0.000992,0.000062,0.000000,0.000000,0.0,NaN,...,0.000000,0.000226,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000
2021-07-26,0.000000,0.0,NaN,0.000000,0.003799,0.000498,0.002026,0.000241,0.0,NaN,...,0.001500,0.000358,0.003711,0.0,0.000000,0.000522,0.001810,0.0,0.001578,0.000000
2021-07-27,0.000784,0.0,NaN,0.000660,0.001646,0.000389,-0.000289,0.000000,0.0,NaN,...,0.001577,0.000773,0.002876,0.0,0.028571,0.000782,0.001314,0.0,-0.008361,0.013055


In [76]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths_pct.csv')
jh_hist_usstates_deaths_pct.to_csv(filename, index=True)

##### 1D-10: Daily % change JH historic **deaths**, grouped / subtotalled by **US County**

In [77]:
jh_hist_uscounties_deaths_pct = jh_hist_uscounties_deaths.pct_change()

In [78]:
jh_hist_uscounties_deaths_pct.tail(3)

Province_State  Alabama                                                       \
Admin2          Autauga   Baldwin Barbour Bibb Blount Bullock Butler Calhoun   
2021-07-25      0.00000  0.000000     0.0  0.0    0.0     0.0    0.0     0.0   
2021-07-26      0.00000  0.000000     0.0  0.0    0.0     0.0    0.0     0.0   
2021-07-27      0.00885  0.003077     0.0  0.0    0.0     0.0    0.0     0.0   

Province_State                    ...   Wyoming                            \
Admin2         Chambers Cherokee  ...      Park Platte Sheridan  Sublette   
2021-07-25          0.0      0.0  ...  0.000000    0.0      0.0  0.000000   
2021-07-26          0.0      0.0  ...  0.000000    0.0      0.0  0.000000   
2021-07-27          0.0      0.0  ...  0.030303    0.0      0.0  0.142857   

Province_State                                                        
Admin2         Sweetwater Teton     Uinta Unassigned Washakie Weston  
2021-07-25       0.000000   0.0  0.000000        NaN      0.0    0.0  
2021-07-26       0.000000   0.0  0.000000        NaN      0.0    0.0  
2021-07-27       0.023256   0.0  0.076923        NaN      0.0    0.0  

[3 rows x 3336 columns]

In [79]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths_pct.csv')
jh_hist_uscounties_deaths_pct.to_csv(filename, index=True)

#### 1E: Calculate exponential weighted moving average (3 day) for each historic dataset (match numbering sequence, export as well)

This is useful to see which countries / states / counties are becoming hot spots or which are cooling down.  
Exponential weighted moving averages (EWMA) blend the most recent x days, using a heavier weight for more recent data.  
https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.ewm.html  
Using the EWMA gives the best balance of minimizing erratic daily changes by using a moving avg, while giving more credence to recent data vs a simple moving avg.  

##### 1E-4: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [80]:
jh_hist_countries_cases_pct_ewma = jh_hist_countries_cases_pct.ewm(span=3).mean()

In [81]:
jh_hist_countries_cases_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-07-25,0.000879,0.000187,0.008085,0.000737,0.002340,0.000729,0.002118,0.000871,0.004988,0.000539,...,0.005698,0.000432,0.005996,4.207258e-30,0.003833,0.081391,0.000242,0.000580,0.003932,0.013725
2021-07-26,0.003388,0.000154,0.008683,0.003093,0.002355,0.000365,0.002354,0.000680,0.005275,0.000507,...,0.005027,0.000417,0.005834,2.103629e-30,0.003746,0.066265,0.000121,0.000290,0.002604,0.017333
2021-07-27,0.004200,0.000194,0.009059,0.001855,0.003935,0.003307,0.002901,0.000733,0.005749,0.000533,...,0.004549,0.000483,0.006073,1.051815e-30,0.003634,0.083788,0.000220,0.000858,0.002966,0.017507


In [82]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases_pct_ewma.csv')
jh_hist_countries_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-5: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **Country**

In [83]:
jh_hist_countries_deaths_pct_ewma = jh_hist_countries_deaths_pct.ewm(span=3).mean()

In [84]:
jh_hist_countries_deaths_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-07-25,0.001668,4.032747e-13,0.004645,4.274469e-28,0.002275,0.005952,0.001956,0.000767,0.000626,0.000030,...,0.000414,0.000969,0.007239,0.0,0.005459,0.006685,0.000614,0.000254,0.007808,0.021441
2021-07-26,0.010272,2.016373e-13,0.005276,2.137235e-28,0.004694,0.002976,0.002829,0.000710,0.002492,0.000061,...,0.000261,0.000906,0.007826,0.0,0.005164,0.211451,0.000307,0.000127,0.007419,0.023487
2021-07-27,0.008070,1.008187e-13,0.005697,1.068617e-28,0.003861,0.001488,0.002601,0.001118,0.000704,0.000264,...,0.000637,0.000874,0.007489,0.0,0.004719,0.105725,0.000292,0.000428,0.006896,0.028605


In [85]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths_pct_ewma.csv')
jh_hist_countries_deaths_pct_ewma.to_csv(filename, index=True)

##### 1E-6: EWMA of Daily % change Export the JH historic **recovered**, grouped / subtotalled by **Country**

In [86]:
jh_hist_countries_recovered_pct_ewma = jh_hist_countries_recovered_pct.ewm(span=3).mean()

In [87]:
jh_hist_countries_recovered_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-07-25,1.809685e-06,0.000086,0.006017,0.000330,0.002841,0.000824,0.003264,0.000540,0.000214,0.000385,...,0.014247,0.001034,0.005552,1.009742e-28,0.004085,0.112743,0.000443,0.000220,0.007316,0.023209
2021-07-26,9.048424e-07,0.000093,0.006377,0.003346,0.004409,0.000412,0.003206,0.000512,0.000253,0.000404,...,0.015902,0.001000,0.006194,5.048710e-29,0.004196,0.108124,0.000221,0.000110,0.006485,0.026124
2021-07-27,4.524212e-07,0.000100,0.006458,0.002952,0.002989,0.000206,0.002910,0.000511,0.000231,0.000451,...,0.017711,0.000859,0.006743,2.524355e-29,0.004455,0.091590,0.000165,0.000295,0.005621,0.032737


In [88]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered_pct_ewma.csv')
jh_hist_countries_recovered_pct_ewma.to_csv(filename, index=True)

##### 1E-7: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [89]:
jh_hist_usstates_cases_pct_ewma = jh_hist_usstates_cases_pct.ewm(span=3).mean()

In [90]:
jh_hist_usstates_cases_pct_ewma.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2021-07-25,0.002889,0.001124,NaN,0.001601,0.003971,0.001240,0.000297,0.000168,0.000650,1.663965e-150,...,0.000402,0.001169,0.000236,0.000285,0.004006,0.000257,0.000542,0.000229,0.000167,0.000494
2021-07-26,0.002674,0.004389,NaN,0.001586,0.002812,0.002107,0.001728,0.000920,0.000618,8.319827e-151,...,0.002260,0.001536,0.002779,0.001602,0.005203,0.001496,0.003566,0.001339,0.000790,0.001803
2021-07-27,0.003668,0.002194,NaN,0.001596,0.004135,0.002367,0.001626,0.001152,0.000675,4.159913e-151,...,0.002319,0.002803,0.002104,0.001043,0.007939,0.001353,0.002875,0.001239,0.001184,0.002236


In [91]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases_pct_ewma.csv')
jh_hist_usstates_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-8: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [92]:
jh_hist_uscounties_cases_pct_ewma = jh_hist_uscounties_cases_pct.ewm(span=3).mean()

In [93]:
jh_hist_uscounties_cases_pct_ewma.tail(3)

Province_State   Alabama                                                    \
Admin2           Autauga   Baldwin   Barbour      Bibb    Blount   Bullock   
2021-07-25      0.001713  0.006556  0.002003  0.003553  0.001378  0.000426   
2021-07-26      0.001935  0.005630  0.001618  0.004097  0.001455  0.000213   
2021-07-27      0.002380  0.008897  0.003681  0.004714  0.003368  0.001699   

Province_State                                          ...   Wyoming  \
Admin2            Butler   Calhoun  Chambers  Cherokee  ...      Park   
2021-07-25      0.003176  0.002190  0.002901  0.002216  ...  0.000194   
2021-07-26      0.002435  0.002020  0.002099  0.001108  ...  0.001118   
2021-07-27      0.003543  0.002725  0.004031  0.002101  ...  0.002086   

Province_State                                                               \
Admin2            Platte  Sheridan  Sublette Sweetwater     Teton     Uinta   
2021-07-25      0.002056  0.000050  0.001116   0.000554  0.000372  0.000624   
2021-07-26      0.003058  0.001089  0.002945   0.001795  0.001867  0.002420   
2021-07-27      0.008267  0.001909  0.005629   0.002209  0.001191  0.001630   

Province_State                                 
Admin2         Unassigned  Washakie    Weston  
2021-07-25           -1.0  0.000887  0.000572  
2021-07-26           -1.0  0.000974  0.004006  
2021-07-27           -1.0  0.001016  0.003480  

[3 rows x 3336 columns]

In [94]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases_pct_ewma.csv')
jh_hist_uscounties_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-9: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **US State**

In [95]:
jh_hist_usstates_deaths_pct_ewma = jh_hist_usstates_deaths_pct.ewm(span=3).mean()

In [96]:
jh_hist_usstates_deaths_pct_ewma.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2021-07-25,0.000346,0.000737,NaN,0.000491,0.001156,0.000117,0.000255,0.000030,0.000074,NaN,...,0.000188,0.000409,0.000116,0.000030,0.004276,0.000108,0.000599,0.000397,0.000177,0.000125
2021-07-26,0.000173,0.000368,NaN,0.000245,0.002477,0.000308,0.001141,0.000136,0.000037,NaN,...,0.000844,0.000384,0.001914,0.000015,0.002138,0.000315,0.001204,0.000199,0.000878,0.000062
2021-07-27,0.000478,0.000184,NaN,0.000453,0.002061,0.000348,0.000426,0.000068,0.000019,NaN,...,0.001211,0.000578,0.002395,0.000008,0.015355,0.000549,0.001259,0.000099,-0.003742,0.006559


In [97]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths_pct_ewma.csv')
jh_hist_usstates_deaths_pct_ewma.to_csv(filename, index=True)

##### 1E-10: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **US County**

In [98]:
jh_hist_uscounties_deaths_pct_ewma = jh_hist_uscounties_deaths_pct.ewm(span=3).mean()

In [99]:
jh_hist_uscounties_deaths_pct_ewma.tail(3)

Province_State       Alabama                                                  \
Admin2               Autauga   Baldwin   Barbour          Bibb        Blount   
2021-07-25      1.811337e-17  0.000825  0.004167  2.980232e-08  1.553319e-24   
2021-07-26      9.056683e-18  0.000413  0.002083  1.490116e-08  7.766595e-25   
2021-07-27      4.424779e-03  0.001745  0.001042  7.450581e-09  3.883298e-25   

Province_State                                                        ...  \
Admin2               Bullock    Butler   Calhoun  Chambers  Cherokee  ...   
2021-07-25      5.165158e-24  0.001761  0.000047  0.001010  0.002660  ...   
2021-07-26      2.582579e-24  0.000880  0.000024  0.000505  0.001330  ...   
2021-07-27      1.291290e-24  0.000440  0.000012  0.000253  0.000665  ...   

Province_State       Wyoming                                                   \
Admin2                  Park    Platte      Sheridan      Sublette Sweetwater   
2021-07-25      3.029903e-08  0.000009  6.123612e-42  2.690158e-62   0.000765   
2021-07-26      1.514951e-08  0.000005  3.061806e-42  1.345079e-62   0.000383   
2021-07-27      1.515152e-02  0.000002  1.530903e-42  7.142857e-02   0.011819   

Province_State                                                       \
Admin2                 Teton         Uinta Unassigned      Washakie   
2021-07-25      1.578984e-15  8.616465e-27   0.752941  1.671440e-54   
2021-07-26      7.894919e-16  4.308232e-27   0.752941  8.357202e-55   
2021-07-27      3.947460e-16  3.846154e-02   0.752941  4.178601e-55   

Province_State                
Admin2                Weston  
2021-07-25      1.421085e-15  
2021-07-26      7.105427e-16  
2021-07-27      3.552714e-16  

[3 rows x 3336 columns]

In [100]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths_pct_ewma.csv')
jh_hist_uscounties_deaths_pct_ewma.to_csv(filename, index=True)

### Dataset 2: US States - Covid Tracking Project

The Covid Tracking Project stores data in both API (JSON) and CSV formats.  We'll be loading the CSV files into Pandas here.  
https://covidtracking.com/api/  

States current - /api/v1/states/current.csv | [CSV](https://covidtracking.com/api/v1/states/current.csv)  
States historic - /api/v1/states/daily.csv | [CSV](https://covidtracking.com/api/v1/states/daily.csv)  
States info - /api/v1/states/info.csv | [CSV](https://covidtracking.com/api/v1/states/info.csv)   

#### 2A: Pull the Data from CT's API, immediately export as 'orig'

In [101]:
ct_live_usstates = pd.read_csv("https://covidtracking.com/api/v1/states/current.csv")
ct_live_usstates.head()

,date,state,positive,probableCases,negative,pending,totalTestResultsSource,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,...,dataQualityGrade,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
0,20210307,AK,56886,NaN,NaN,NaN,totalTestsViral,1731628,33.0,1293.0,...,NaN,0,0,dc4bccd4bb885349d7e94d6fed058e285d4be164,0,0,0,0,0,NaN
1,20210307,AL,499819,107742.0,1931711.0,NaN,totalTestsPeopleViral,2323788,494.0,45976.0,...,NaN,-1,0,997207b430824ea40b8eb8506c19a93e07bc972e,0,0,0,0,0,NaN
2,20210307,AR,324818,69092.0,2480716.0,NaN,totalTestsViral,2736442,335.0,14926.0,...,NaN,22,11,50921aeefba3e30d31623aa495b47fb2ecc72fae,0,0,0,0,0,NaN
3,20210307,AS,0,NaN,2140.0,NaN,totalTestsViral,2140,NaN,NaN,...,NaN,0,0,f77912d0b80d579fbb6202fa1a90554fc4dc1443,0,0,0,0,0,NaN
4,20210307,AZ,826454,56519.0,3073010.0,NaN,totalTestsViral,7908105,963.0,57907.0,...,NaN,5,44,0437a7a96f4471666f775e63e86923eb5cbd8cdf,0,0,0,0,0,NaN


In [102]:
filename = os.path.join(os.getcwd(), 'Datasets/CT/orig/ct_live_usstates_orig.csv')
ct_live_usstates.to_csv(filename, index=False)

In [103]:
ct_hist_usstates = pd.read_csv("https://covidtracking.com/api/v1/states/daily.csv")
ct_hist_usstates.head()

,date,state,positive,probableCases,negative,pending,totalTestResultsSource,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,...,dataQualityGrade,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
0,20210307,AK,56886.0,NaN,NaN,NaN,totalTestsViral,1731628.0,33.0,1293.0,...,NaN,0,0,dc4bccd4bb885349d7e94d6fed058e285d4be164,0,0,0,0,0,NaN
1,20210307,AL,499819.0,107742.0,1931711.0,NaN,totalTestsPeopleViral,2323788.0,494.0,45976.0,...,NaN,-1,0,997207b430824ea40b8eb8506c19a93e07bc972e,0,0,0,0,0,NaN
2,20210307,AR,324818.0,69092.0,2480716.0,NaN,totalTestsViral,2736442.0,335.0,14926.0,...,NaN,22,11,50921aeefba3e30d31623aa495b47fb2ecc72fae,0,0,0,0,0,NaN
3,20210307,AS,0.0,NaN,2140.0,NaN,totalTestsViral,2140.0,NaN,NaN,...,NaN,0,0,f77912d0b80d579fbb6202fa1a90554fc4dc1443,0,0,0,0,0,NaN
4,20210307,AZ,826454.0,56519.0,3073010.0,NaN,totalTestsViral,7908105.0,963.0,57907.0,...,NaN,5,44,0437a7a96f4471666f775e63e86923eb5cbd8cdf,0,0,0,0,0,NaN


In [104]:
filename = os.path.join(os.getcwd(), 'Datasets/CT/orig/ct_hist_usstates_orig.csv')
ct_hist_usstates.to_csv(filename, index=False)

#### 2B: Change the dates to datetime format, make them the index for both datasets

In [105]:
ct_live_usstates['date'] = ct_live_usstates['date'].map(lambda d : str(d)[:4] + '-' + str(d)[4:6] + '-' + str(d)[6:])
ct_live_usstates['date'] = pd.to_datetime(ct_live_usstates['date'])
ct_live_usstates.set_index('date', inplace=True)
ct_live_usstates.head()

,state,positive,probableCases,negative,pending,totalTestResultsSource,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,...,dataQualityGrade,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
date,,,,,,,,,,,,,,,,,,,,,
2021-03-07,AK,56886,NaN,NaN,NaN,totalTestsViral,1731628,33.0,1293.0,NaN,...,NaN,0,0,dc4bccd4bb885349d7e94d6fed058e285d4be164,0,0,0,0,0,NaN
2021-03-07,AL,499819,107742.0,1931711.0,NaN,totalTestsPeopleViral,2323788,494.0,45976.0,NaN,...,NaN,-1,0,997207b430824ea40b8eb8506c19a93e07bc972e,0,0,0,0,0,NaN
2021-03-07,AR,324818,69092.0,2480716.0,NaN,totalTestsViral,2736442,335.0,14926.0,141.0,...,NaN,22,11,50921aeefba3e30d31623aa495b47fb2ecc72fae,0,0,0,0,0,NaN
2021-03-07,AS,0,NaN,2140.0,NaN,totalTestsViral,2140,NaN,NaN,NaN,...,NaN,0,0,f77912d0b80d579fbb6202fa1a90554fc4dc1443,0,0,0,0,0,NaN
2021-03-07,AZ,826454,56519.0,3073010.0,NaN,totalTestsViral,7908105,963.0,57907.0,273.0,...,NaN,5,44,0437a7a96f4471666f775e63e86923eb5cbd8cdf,0,0,0,0,0,NaN


In [106]:
ct_hist_usstates['date'] = ct_hist_usstates['date'].map(lambda d : str(d)[:4] + '-' + str(d)[4:6] + '-' + str(d)[6:])
ct_hist_usstates['date'] = pd.to_datetime(ct_hist_usstates['date'])
ct_hist_usstates.set_index('date', inplace=True)
ct_hist_usstates.head()

,state,positive,probableCases,negative,pending,totalTestResultsSource,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,...,dataQualityGrade,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
date,,,,,,,,,,,,,,,,,,,,,
2021-03-07,AK,56886.0,NaN,NaN,NaN,totalTestsViral,1731628.0,33.0,1293.0,NaN,...,NaN,0,0,dc4bccd4bb885349d7e94d6fed058e285d4be164,0,0,0,0,0,NaN
2021-03-07,AL,499819.0,107742.0,1931711.0,NaN,totalTestsPeopleViral,2323788.0,494.0,45976.0,NaN,...,NaN,-1,0,997207b430824ea40b8eb8506c19a93e07bc972e,0,0,0,0,0,NaN
2021-03-07,AR,324818.0,69092.0,2480716.0,NaN,totalTestsViral,2736442.0,335.0,14926.0,141.0,...,NaN,22,11,50921aeefba3e30d31623aa495b47fb2ecc72fae,0,0,0,0,0,NaN
2021-03-07,AS,0.0,NaN,2140.0,NaN,totalTestsViral,2140.0,NaN,NaN,NaN,...,NaN,0,0,f77912d0b80d579fbb6202fa1a90554fc4dc1443,0,0,0,0,0,NaN
2021-03-07,AZ,826454.0,56519.0,3073010.0,NaN,totalTestsViral,7908105.0,963.0,57907.0,273.0,...,NaN,5,44,0437a7a96f4471666f775e63e86923eb5cbd8cdf,0,0,0,0,0,NaN
